In [1]:
from IPython.core.display import display, HTML
import yaml

In [2]:
thingToUpdate = 'invalid'
while thingToUpdate not in ['plugins', 'themes']:
    thingToUpdate = input("What do you want to update? (plugins or themes)?")

What do you want to update? (plugins or themes)? themes


In [3]:
with open(f'all_{thingToUpdate}_info.yml', 'r') as file:
    originalEntryInfo = yaml.load(file, Loader=yaml.FullLoader)

In [4]:
def describe_entry(entry):
    properties = []
    for key, value in entry.items():
        if key in ['description_url', 'download_url']:
            properties.append(f'{key.capitalize()}: <a href="{value}">{value}</a>')
        else:
            properties.append(f'{key.capitalize()}: {value}')
    description = '<br/>'.join(properties)
    return description

In [5]:
for name, entry in originalEntryInfo.items():
    if 'available' not in entry:
        display(HTML(describe_entry(entry)))
        available = 'invalid'
        while available not in ['Y', 'N']:
            available = input("Make this plugin available? (Y/N)")
        note = input("Add any note about the plugin (justification for availablility)")
        if available == 'Y':
            originalEntryInfo[name]['available'] = True
        else:
            originalEntryInfo[name]['available'] = False
        originalEntryInfo[name]['note'] = note
        with open(f'all_{thingToUpdate}_info.yml', 'w') as file:
            documents = yaml.dump(originalEntryInfo, file)

Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


...

Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works, default


Make this plugin available? (Y/N) Y
Add any note about the plugin (justification for availablility) Works


In [6]:
with open(f'{thingToUpdate}.yml', 'r') as file:
    cache_config = yaml.load(file, Loader=yaml.FullLoader)

In [8]:
for name, entry in originalEntryInfo.items():
    if entry['available']:
        if name in cache_config:
            # Plugin exists in cache config
            if cache_config[name]['url'] != entry['download_url']:
                # Download urls differ, need input on which to use
                print(f"Addon download links differ.\nOld: {cache_config[name]['url']}\nNew: {entry['download_url']}")
                do_update = 'invalid'
                while do_update not in ['Y', 'N']:
                    do_update = input("Replace the old link with the new one? (Y/N)")
                if do_update == 'Y':
                    cache_config[name]['url'] = entry['download_url']
                continue
            else:
                # Download urls are the same, nothing to do
                continue
        else:
            # Plugin does not exist in cache config, but is listed as available. Need input on name parameter.
            print(f"Please provide a name for {entry['name']}")
            plugin_folder_name = input("Name for addon folder (if for plugin, needs to match main plugin php file)")
            cache_config[name] = {
                'name': plugin_folder_name,
                'url': entry['download_url']
            }
            continue
    else:
        # Plugin is not listed as available, nothing to do
        continue

Please provide a name for BigStuff


Name for addon folder (if for plugin, needs to match main plugin php file) BigStuff


Addon download links differ.
Old: https://github.com/omeka/theme-centerrow/releases/download/v1.4.1/centerrow-v1.4.1.zip
New: https://github.com/omeka/theme-centerrow/releases/download/v1.5.1/theme-centerrow-v1.5.1.zip


Replace the old link with the new one? (Y/N) Y


Addon download links differ.
Old: https://github.com/omeka/theme-emiglio/releases/download/v2.3/theme-emiglio-v2.3.zip
New: https://github.com/omeka/theme-emiglio/releases/download/v2.4/theme-emiglio-v2.4.zip


...

Replace the old link with the new one? (Y/N) Y


Please provide a name for Thanks, Roy


Name for addon folder (if for plugin, needs to match main plugin php file) default


Addon download links differ.
Old: https://github.com/omeka/theme-thedaily/releases/download/v1.4.1/thedaily-v1.4.1.zip
New: https://github.com/omeka/theme-thedaily/releases/download/v1.5.1/theme-thedaily-v1.5.1.zip


Replace the old link with the new one? (Y/N) Y


In [9]:
with open(f'{thingToUpdate}.yml', 'w') as file:
    documents = yaml.dump(cache_config, file)